In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.

In [ ]:
!rm -rf /root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat

In [1]:
!pip install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python

  Cloning https://github.com/vchiley/triton.git (to revision triton_pre_mlir) to /tmp/pip-install-2ovpsit6/triton-pre-mlir_7058a036da0f4bf188ac14f85fcd398a
  Running command git clone --filter=blob:none --quiet https://github.com/vchiley/triton.git /tmp/pip-install-2ovpsit6/triton-pre-mlir_7058a036da0f4bf188ac14f85fcd398a
  Running command git checkout -b triton_pre_mlir --track origin/triton_pre_mlir
  Switched to a new branch 'triton_pre_mlir'
  Branch 'triton_pre_mlir' set up to track remote branch 'triton_pre_mlir' from 'origin'.
  Resolved https://github.com/vchiley/triton.git to commit 2dd3b957698a39bbca615c02a447a98482c144a3
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for triton-pre-mlir: filename=triton_pre_mlir-2.0.0-cp311-cp311-linux_x86_64.whl size=15433811 sha256=ce8526b1eacf05ee08cd889830957e1523471403b35c020f596586b939196054
  Stored in directory: /tmp/pip-ephem-wheel-cache-gkgdvp67/wheels/33/6e/9d/

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model
)
# 1. Model and Tokenizer Setup
model_id = "vinai/PhoGPT-4B-Chat"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=torch.float16
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


hf_prefixlm_converter.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- modeling_mpt.py
- adapt_tokenizer.py
- custom_embedding.py
- meta_init_context.py
- param_init_fns.py
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

In [6]:
for name, _ in model.named_modules():
    if "attn" in name.lower():
        print(name)


transformer.blocks.0.attn
transformer.blocks.0.attn.Wqkv
transformer.blocks.0.attn.out_proj
transformer.blocks.0.resid_attn_dropout
transformer.blocks.1.attn
transformer.blocks.1.attn.Wqkv
transformer.blocks.1.attn.out_proj
transformer.blocks.1.resid_attn_dropout
transformer.blocks.2.attn
transformer.blocks.2.attn.Wqkv
transformer.blocks.2.attn.out_proj
transformer.blocks.2.resid_attn_dropout
transformer.blocks.3.attn
transformer.blocks.3.attn.Wqkv
transformer.blocks.3.attn.out_proj
transformer.blocks.3.resid_attn_dropout
transformer.blocks.4.attn
transformer.blocks.4.attn.Wqkv
transformer.blocks.4.attn.out_proj
transformer.blocks.4.resid_attn_dropout
transformer.blocks.5.attn
transformer.blocks.5.attn.Wqkv
transformer.blocks.5.attn.out_proj
transformer.blocks.5.resid_attn_dropout
transformer.blocks.6.attn
transformer.blocks.6.attn.Wqkv
transformer.blocks.6.attn.out_proj
transformer.blocks.6.resid_attn_dropout
transformer.blocks.7.attn
transformer.blocks.7.attn.Wqkv
transformer.blocks.

In [7]:
# 2. Prepare Model for QLoRA
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

# 3. Configure LoRA
lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["attn.Wqkv", "attn.out_proj"]  # targeting attention components
)
model = get_peft_model(model, lora_config)

In [8]:
model.print_trainable_parameters()

trainable params: 9,437,184 || all params: 3,697,514,496 || trainable%: 0.255230480102491


In [9]:
import json
from datasets import Dataset
def format_conversation(prompt, response):
    # Format the conversation in the style expected by the model
    return f"Human: {prompt}\nAssistant: {response}"
def prepare_dataset(data_path):
    # Load your JSON data
    with open(data_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            print("Please check if the JSON file is correctly formatted.")
            return None

    # Ensure the loaded data is a list
    if not isinstance(data, list):
        print("JSON data is not a list of objects. Please check the file format.")
        return None

    # Format the data according to the YAML config
    formatted_data = []
    for item in data:
        # Skip items that have qa_pairs
        if isinstance(item, dict) and "qa_pairs" in item:
            continue

        # Process items that don't have qa_pairs
        if isinstance(item, dict):
            if "prompt" in item and "response" in item:
                conversation = {
                    "text": format_conversation(item["prompt"], item["response"])
                }
                formatted_data.append(conversation)
            else:
                print(f"Skipping item due to missing 'prompt' or 'response' key: {item}")
        else:
            print(f"Skipping item as it is not a dictionary: {item}")

    # Return Dataset.from_list only if formatted_data is not empty
    if formatted_data:
        return Dataset.from_list(formatted_data)
    else:
        print("No valid data found to create a dataset.")
        return None

# Save dataset to HuggingFace format
dataset = prepare_dataset("/content/combined_qa_dataset.json")

# Only save to disk if the dataset was successfully created
if dataset is not None:
    dataset.save_to_disk("/content/sample_instruction_following_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1338 [00:00<?, ? examples/s]

In [10]:
from datasets import load_from_disk

# Load the dataset we just prepared
dataset = load_from_disk("/content/sample_instruction_following_dataset")

In [13]:
print(dataset[0])


{'text': 'Human: Công tác tư vấn học tập và hướng nghiệp của Trường Đại học Quốc tế bao gồm những gì?\nAssistant: Công tác tư vấn học tập và hướng nghiệp bao gồm tư vấn, hỗ trợ sinh viên trong việc xây dựng kế hoạch học tập, phương pháp học tập và cung cấp thông tin về chương trình đào tạo.'}


In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Configure training arguments without evaluation
training_args = TrainingArguments(
    output_dir="./phogpt-lora-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    # Remove evaluation_strategy and load_best_model_at_end
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_grad_norm=0.3,
)

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",  # This should match the field name in your dataset
    max_seq_length=512,
    args=training_args
)

# Start training
trainer.train()

Map:   0%|          | 0/1338 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


Step,Training Loss
10,1.631400
20,1.523200
30,1.426900
40,1.408400
50,1.346600
60,1.335500
70,1.304600
80,1.267900
90,1.255400
100,1.249800


/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passi

TrainOutput(global_step=249, training_loss=1.2382368141388798, metrics={'train_runtime': 490.1875, 'train_samples_per_second': 8.189, 'train_steps_per_second': 0.508, 'total_flos': 8155780565188608.0, 'train_loss': 1.2382368141388798, 'epoch': 2.97})

In [19]:
base_model = AutoModelForCausalLM.from_pretrained(
    "vinai/PhoGPT-4B-Chat",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
# 2. Load your trained LoRA adapter
model = PeftModel.from_pretrained(base_model, "./phogpt-lora-finetuned/checkpoint-249")

# 3. Merge and unload
model = model.merge_and_unload()

# 4. Save merged model
model.save_pretrained("./phogpt-merged")
tokenizer.save_pretrained("./phogpt-merged")

('./phogpt-merged/tokenizer_config.json',
 './phogpt-merged/special_tokens_map.json',
 './phogpt-merged/tokenizer.json')

In [20]:
# Load the merged model
model = AutoModelForCausalLM.from_pretrained(
    "./phogpt-merged",
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("./phogpt-merged", trust_remote_code=True)

In [21]:
model.tie_weights()

In [24]:
# Format using the chat template (add_generation_prompt=True adds assistant tag)
query = "Sinh viên bị đình chỉ trong trường hợp nào?"

# Manually format the prompt string to match the training format
# "Human: {prompt}\nAssistant: {response}" was used in prepare_dataset
# For inference, we only need "Human: {prompt}\nAssistant:"
prompt = f"Human: {query}\nAssistant:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate the response
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and clean up the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Remove the original prompt from the response
# Find the start of the Assistant's response
assistant_start_index = response.find("Assistant:")
if assistant_start_index != -1:
    response = response[assistant_start_index + len("Assistant:"):].strip()
else:
    # Handle cases where 'Assistant:' might not be in the output
    response = response.strip()


print("✅ Response:", response)

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


✅ Response: Sinh viên bị đình chỉ học tập vì vi phạm nội quy, quy định nhà trường. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-43]. [cite: 42-4
